## Project

In [ ]:
import gurobipy as gp
import numpy as np
import math

In [35]:
# Define dimensions of optimization problem

I = 3 # number of Customer
J = 5 # Number of Supplier
T = 4 # number of time periods

In [36]:
# Define parameter values
c = [[550, 480, 420, 890, 790],[570, 85, 450, 235, 525],[825, 430, 850, 390, 720]]
p = [[66, 73, 62, 60, 71],[121, 134, 127, 142, 122],[28, 26, 29, 25, 23]]
s0 = [[0.95, 0.97, 0.94, 0.99, 0.98],[0.92, 0.99, 0.91, 0.99, 0.92],[0.99, 0.98, 0.99, 0.97, 0.99]]
f0 = [[0.95, 0.98, 0.92, 0.96, 0.99],[0.89, 0.92, 0.95, 0.90, 0.91],[0.93, 0.87, 0.88, 0.83, 0.87]]
gamma = [[0.0013, 0.0011, 0.001, 0.001, 0.0011],[0.00, 0.0014, 0.002, 0.0015, 0.002],[0.0011, 0.0016, 0.001, 0.002, 0.0011]]
Lambda = [[0.001, 0.002, 0.0015, 0.001, 0.001],[0.00, 0.0015, 0.001, 0.002, 0.002],[0.001, 0.002, 0.0015, 0.0015, 0.0012]]
d = [[454, 540, 675, 755],[327, 320, 290, 285],[645, 650, 637, 663]]
beta = [0.08, 0.1, 0.09, 0.12, 0.09]
o = [45000, 64500, 37250, 53400, 42250]
v = [85, 150, 85, 100, 150]
g = [33500, 55200, 33500, 38400, 55200]
h = [12, 35, 7]
w = [0.75, 0.85, 0.60]
b = [17, 38, 10]
W = 150

In [46]:
# Define the decision variables

x = {} # order quantity for each product in each period
y = {} # binary variable for each product in each period
z = {} # continuous variable for each product in each period
model = gp.Model("Project")

# Loop through products, periods, and time intervals
for j in range(J):
    for t in range(T):
        for i in range(I):
            
            # Add decision variables for order quantity with lower and upper bounds
            x[i,j,t] = model.addVar(vtype=gp.GRB.INTEGER,name="x{}_{}_{}".format(i+1,j+1,t+1), lb = 0, ub = c[i][j])
         
        # Add binary variable for each product in each period
        y[j,t] = model.addVar(vtype=gp.GRB.BINARY, name="y{}_{}".format(j + 1, t + 1))
       
    # Add continuous variable for each product in each period
    # z[j,t] = model.addVar(vtype=gp.GRB.CONTINUOUS, name="z{}_{}".format (j+1, t+1))


In [67]:
# Define Objective Function (z1)
z1_a = gp.quicksum(p[i][j]*x[i,j,t] for i in range(I) for j in range(J) for t in range(T))
z1_b = gp.quicksum(o[j]*math.exp(-beta[j])*y[j,t] for j in range(J) for t in range(T))
z1_c = gp.quicksum(h[i]*(gp.quicksum(x[i,j,k] for j in range(J) for k in range(t))-gp.quicksum((1-s0[i][j]*math.exp(gamma[i][j]*t)*x[i,j,t]) for j in range(J))-gp.quicksum(d[i][k] for k in range(t))) for i in range(I) for t in range(T))
z1_d = gp.quicksum(g[j]*(gp.quicksum(w[i]*x[i,j,t]/v[j] for i in range(I))) for j in range(J) for t in range(T))
z1_obj = z1_a + z1_b + z1_c + z1_d

In [66]:
#Define Objective functions (z2) and (z3)
z2_obj = gp.quicksum(f0[i][j]*math.exp(Lambda[i][j]*t)*x[i,j,t] for i in range(I) for j in range(J) for t in range(T))
z3_obj = gp.quicksum(s0[i][j]*math.exp(gamma[i][j]*t)*x[i,j,t] for i in range(I) for j in range(J) for t in range(T))

In [74]:
# Define the objective function for the model
model.setObjectiveN(z1_obj, index=1, priority=0) # minimize z1
model.setObjectiveN(-z2_obj, index=2, priority=1) # maximize -z2
model.setObjectiveN(-z3_obj, index=3, priority=1) # maximize -z3

# Set the optimization sense to minimize
model.ModelSense = gp.GRB.MINIMIZE


In [50]:
# Define Constraints

# Constraint (14)
for i in range(I):
    for t in range(T):
        model.addConstr(gp.quicksum(x[i,j,k] for k in range(t) for j in range(J))-gp.quicksum(d[i][k] for k in range(t)) >= gp.quicksum((1-s0[i][j]*math.exp(gamma[i][j]*t) for j in range(J)))*x[i,j,t])
        
# Constraint (15)
for i in range(I):
    for t in range(T):
        for j in range(J):
            model.addConstr(gp.quicksum(d[i][k] for k in range(t,T))*y[j,t]-x[i,j,t] >= 0)
            
# Constraint (16)
for i in range(I):
    model.addConstr(gp.quicksum(x[i,j,t] for t in range(T) for j in range(J))-gp.quicksum((1-s0[i][j]*math.exp(gamma[i][j]*T))*x[i,j,T-1] for j in range(J)) - gp.quicksum(d[i][t] for t in range(T)) <= 0.5)
    model.addConstr(gp.quicksum(x[i,j,k] for k in range(T) for j in range(J))-gp.quicksum((1-s0[i][j]*math.exp(gamma[i][j]*T))*x[i,j,T-1] for j in range(J)) - gp.quicksum(d[i][t] for t in range(T)) >= 0.0)

# Constraint (17)    
for t in range(T):
    model.addConstr(gp.quicksum(w[i]*(gp.quicksum(x[i,j,k] for k in range(t) for j in range(J))-gp.quicksum((1-s0[i][j]*math.exp(gamma[i][j]*t))*x[i,j,t] for j in range(J))-gp.quicksum(d[i][k] for k in range(t))) for i in range(I)) <= W)

# Constraint (18)
for i in range(I):
    for j in range(J):
        for t in range(T):
            model.addConstr(x[i, j, t] <= c[i][j], name=f"x_constraint_{j}_{t}")
            
#Constraint (19)
for i in range(I):
    for j in range(J):
        for t in range(T):
            model.addConstr(x[i, j, t] >= 0, name=f"x_constraint_{i}_{j}_{t}")


In [84]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 382 rows, 440 columns and 12972 nonzeros
Model fingerprint: 0x88fd1c7f
Variable types: 360 continuous, 80 integer (20 binary)
Coefficient statistics:
  Matrix range     [4e-03, 6e+04]
  Objective range  [8e-01, 6e+04]
  Bounds range     [1e+00, 9e+02]
  RHS range        [9e+01, 1e+05]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 4 objectives (2 combined) ...
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 282 rows and 38 columns
Presolve time: 0.00s
Presolved: 100 rows and 402 columns
-------------------

In [98]:
for i in range(I):
    for j in range(J):
        for t in range(T):
            print(f"X{i+1}{j+1}{t+1} = {int(x.get((i, j, t)).X)}", end="\t")
        print(" ")
        

X111 = 0	X112 = 0	X113 = 0	X114 = 69	 
X121 = 480	X122 = 0	X123 = 0	X124 = 480	 
X131 = 0	X132 = 0	X133 = 0	X134 = 0	 
X141 = 65	X142 = 0	X143 = 0	X144 = 1	 
X151 = 0	X152 = 552	X153 = 602	X154 = 194	 
X211 = 0	X212 = 0	X213 = 0	X214 = 22	 
X221 = 85	X222 = 85	X223 = 85	X224 = 0	 
X231 = 7	X232 = 0	X233 = 0	X234 = 285	 
X241 = 235	X242 = 235	X243 = 206	X244 = 0	 
X251 = 0	X252 = 0	X253 = 0	X254 = 3	 
X311 = 716	X312 = 579	X313 = 641	X314 = 663	 
X321 = 0	X322 = 0	X323 = 0	X324 = 0	 
X331 = 0	X332 = 0	X333 = 0	X334 = 0	 
X341 = 0	X342 = 0	X343 = 0	X344 = 0	 
X351 = 0	X352 = 0	X353 = 0	X354 = 0	 


In [86]:
for j in range(J):
    for t in range(T):
        print(f"y{j+1}{t+1} = {int(y.get((j, t)).X)}", end="  \t")
    print(" ")


y11 = 1  	y12 = 1  	y13 = 1  	y14 = 1  	 
y21 = 1  	y22 = 1  	y23 = 1  	y24 = 1  	 
y31 = 1  	y32 = 1  	y33 = 1  	y34 = 1  	 
y41 = 1  	y42 = 1  	y43 = 1  	y44 = 1  	 
y51 = 1  	y52 = 1  	y53 = 1  	y54 = 1  	 


In [87]:
print(f"z1 = {z1_obj.getValue():.3f}")
print(f"z2 = {z2_obj.getValue():.3f}")
print(f"z3 = {z3_obj.getValue():.3f}")


z1 = 3085788.447
z2 = 5975.457
z3 = 6177.797


## Goal Programming

In [126]:
# Define priorities, goals, and deviation
priorities = [1, 2, 2]
goals = [0, 0, 0]
weights = [0.5, 0.3, 0.2]
d_pos = {}
d_neg = {}

# Loop through customer, supplier, and time periods
for i, priority in enumerate(priorities):
    for j in range(J):
        for t in range(T):
            
            # Add decision variables for positive and negative deviations
            d_pos[i, j, t] = model.addVar(vtype=gp.GRB.CONTINUOUS, name=f"d_pos_{i}_{j}_{t}")
            d_neg[i, j, t] = model.addVar(vtype=gp.GRB.CONTINUOUS, name=f"d_neg_{i}_{j}_{t}")


In [141]:
# Define the objective function for the model
weighted_obj = gp.quicksum(weights[i] * obj for i, obj in enumerate([z1_obj, z2_obj, z3_obj]))
model.setObjective(weighted_obj, gp.GRB.MINIMIZE)

In [147]:
# Add constraints related to the deviation variables
for i in range(I):
    for j in range(J):
        for t in range(T):
            model.addConstr(z1_obj - goals[0] + d_pos[0, j, t] - d_neg[0, j, t] == 0, name=f"deviation_constraint_1_{j}_{t}")
            model.addConstr(z2_obj - goals[1] + d_pos[1, j, t] - d_neg[1, j, t] == 0, name=f"deviation_constraint_2_{j}_{t}")
            model.addConstr(z3_obj - goals[2] + d_pos[2, j, t] - d_neg[2, j, t] == 0, name=f"deviation_constraint_3_{j}_{t}")

In [148]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1462 rows, 1520 columns and 87132 nonzeros
Model fingerprint: 0x561013aa
Variable types: 1440 continuous, 80 integer (20 binary)
Coefficient statistics:
  Matrix range     [4e-03, 6e+04]
  Objective range  [8e-01, 6e+04]
  Bounds range     [1e+00, 9e+02]
  RHS range        [9e+01, 1e+05]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 4 objectives (2 combined) ...
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 1282 rows and 198 columns
Presolve time: 0.06s
Presolved: 180 rows and 1322 columns
-------------

In [153]:
if model.status == gp.GRB.Status.OPTIMAL:
    print(f"Combined objective value: {model.objVal:.3f}")
else:
    print("No optimal solution found.")

Combined objective value: 1450646.140


In [151]:
# Print the optimal solution
if model.status == gp.GRB.Status.OPTIMAL:
    print("Optimal solution found:")
    for i in range(I):
        for j in range(J):
            for t in range(T):
                print(f"x[{i+1},{j+1},{t+1}] = {int(x[i, j, t].x)}")

else:
    print("No optimal solution found.")


Optimal solution found:
x[1,1,1] = 0
x[1,1,2] = 0
x[1,1,3] = 0
x[1,1,4] = 4
x[1,2,1] = 0
x[1,2,2] = 0
x[1,2,3] = 0
x[1,2,4] = 479
x[1,3,1] = 0
x[1,3,2] = 0
x[1,3,3] = 0
x[1,3,4] = 2
x[1,4,1] = 545
x[1,4,2] = 0
x[1,4,3] = 0
x[1,4,4] = 0
x[1,5,1] = 0
x[1,5,2] = 548
x[1,5,3] = 615
x[1,5,4] = 248
x[2,1,1] = 7
x[2,1,2] = 0
x[2,1,3] = 0
x[2,1,4] = 22
x[2,2,1] = 85
x[2,2,2] = 85
x[2,2,3] = 85
x[2,2,4] = 0
x[2,3,1] = 0
x[2,3,2] = 0
x[2,3,3] = 0
x[2,3,4] = 285
x[2,4,1] = 235
x[2,4,2] = 235
x[2,4,3] = 206
x[2,4,4] = 0
x[2,5,1] = 0
x[2,5,2] = 0
x[2,5,3] = 0
x[2,5,4] = 3
x[3,1,1] = 645
x[3,1,2] = 650
x[3,1,3] = 638
x[3,1,4] = 663
x[3,2,1] = 0
x[3,2,2] = 0
x[3,2,3] = 0
x[3,2,4] = 0
x[3,3,1] = 0
x[3,3,2] = 0
x[3,3,3] = 0
x[3,3,4] = 0
x[3,4,1] = 0
x[3,4,2] = 0
x[3,4,3] = 0
x[3,4,4] = 0
x[3,5,1] = 0
x[3,5,2] = 0
x[3,5,3] = 0
x[3,5,4] = 3


In [152]:
# Print the generated values for y
print("\nGenerated values for y:")
for j in range(J):
    for t in range(T):
        print(f"y[{j+1},{t+1}] = {int(y[j, t].x)}")


Generated values for y:
y[1,1] = 1
y[1,2] = 1
y[1,3] = 1
y[1,4] = 1
y[2,1] = 1
y[2,2] = 1
y[2,3] = 1
y[2,4] = 1
y[3,1] = 0
y[3,2] = 0
y[3,3] = 0
y[3,4] = 1
y[4,1] = 1
y[4,2] = 1
y[4,3] = 1
y[4,4] = 0
y[5,1] = 0
y[5,2] = 1
y[5,3] = 1
y[5,4] = 1


In [146]:
# Print the generated values for d_pos(positive deviation) and d_neg(negative deviation)
print("\nGenerated values for d_pos and d_neg:")
for i, priority in enumerate(priorities):
    for j in range(J):
        for t in range(T):
            print(f"d_pos[{i + 1},{j + 1},{t + 1}] = {d_pos[i, j, t].x:.2f}, d_neg[{i + 1},{j + 1},{t + 1}] = {d_neg[i, j, t].x:.3f}")



Generated values for d_pos and d_neg:
d_pos[1,1,1] = 0.00, d_neg[1,1,1] = 2895237.842
d_pos[1,1,2] = 0.00, d_neg[1,1,2] = 2895237.842
d_pos[1,1,3] = 0.00, d_neg[1,1,3] = 2895237.842
d_pos[1,1,4] = 0.00, d_neg[1,1,4] = 2895237.842
d_pos[1,2,1] = 0.00, d_neg[1,2,1] = 2895237.842
d_pos[1,2,2] = 0.00, d_neg[1,2,2] = 2895237.842
d_pos[1,2,3] = 0.00, d_neg[1,2,3] = 2895237.842
d_pos[1,2,4] = 0.00, d_neg[1,2,4] = 2895237.842
d_pos[1,3,1] = 0.00, d_neg[1,3,1] = 2895237.842
d_pos[1,3,2] = 0.00, d_neg[1,3,2] = 2895237.842
d_pos[1,3,3] = 0.00, d_neg[1,3,3] = 2895237.842
d_pos[1,3,4] = 0.00, d_neg[1,3,4] = 2895237.842
d_pos[1,4,1] = 0.00, d_neg[1,4,1] = 2895237.842
d_pos[1,4,2] = 0.00, d_neg[1,4,2] = 2895237.842
d_pos[1,4,3] = 0.00, d_neg[1,4,3] = 2895237.842
d_pos[1,4,4] = 0.00, d_neg[1,4,4] = 2895237.842
d_pos[1,5,1] = 0.00, d_neg[1,5,1] = 2895237.842
d_pos[1,5,2] = 0.00, d_neg[1,5,2] = 2895237.842
d_pos[1,5,3] = 0.00, d_neg[1,5,3] = 2895237.842
d_pos[1,5,4] = 0.00, d_neg[1,5,4] = 2895237.842
d

In [154]:
# Print the optimal solution
if model.status == gp.GRB.Status.OPTIMAL:
    print(f"Combined objective value: {model.objVal:.2f}")

    # Print decision variables
    for v in model.getVars():
        if v.x > 0:
            print(f"{v.varName}: {v.x}")

else:
    print("No optimal solution found.")

Combined objective value: 1450646.14
x2_1_1: 7.0
x3_1_1: 645.0
y1_1: 1.0
x3_1_2: 650.0
y1_2: 1.0
x3_1_3: 638.0
y1_3: 1.0
x1_1_4: 4.0
x2_1_4: 22.0
x3_1_4: 663.0
y1_4: 1.0
x2_2_1: 85.0
y2_1: 1.0
x2_2_2: 85.0
y2_2: 1.0
x2_2_3: 85.0
y2_3: 1.0
x1_2_4: 479.0
y2_4: 1.0
x1_3_4: 2.0
x2_3_4: 285.0
y3_4: 1.0
x1_4_1: 545.0
x2_4_1: 235.0
y4_1: 1.0
x2_4_2: 235.0
y4_2: 1.0
x2_4_3: 206.0
y4_3: 1.0
x1_5_2: 548.0
y5_2: 1.0
x1_5_3: 615.0
y5_3: 1.0
x1_5_4: 248.0
x2_5_4: 3.0
x3_5_4: 3.0
y5_4: 1.0
d_neg_0_0_0: 2895237.8420754517
d_neg_0_0_1: 2895237.8420754517
d_neg_0_0_2: 2895237.8420754517
d_neg_0_0_3: 2895237.8420754517
d_neg_0_1_0: 2895237.8420754517
d_neg_0_1_1: 2895237.8420754517
d_neg_0_1_2: 2895237.8420754517
d_neg_0_1_3: 2895237.8420754517
d_neg_0_2_0: 2895237.8420754517
d_neg_0_2_1: 2895237.8420754517
d_neg_0_2_2: 2895237.8420754517
d_neg_0_2_3: 2895237.8420754517
d_neg_0_3_0: 2895237.8420754517
d_neg_0_3_1: 2895237.8420754517
d_neg_0_3_2: 2895237.8420754517
d_neg_0_3_3: 2895237.8420754517
d_neg_0